In [1]:
import pandas as pd
import os
import unittest
import string
import re
import copy
pd.set_option('display.max_rows', 10)

In [4]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day20_input.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s = file1.read().splitlines()
pd.Series(s)

0       Tile 3187:
1       .........#
2       ....##...#
3       ##..#.#..#
4       #.##....##
           ...    
1722    ..#.......
1723    ...##.#...
1724    .##.#.....
1725    ##.##....#
1726    ####.#.###
Length: 1727, dtype: object

In [5]:
trainFile = "F:/Code/AOC/2020/inputs/2020_day20_example1.txt"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
file1 = open(trainFile, 'r') 
s_ex1 = file1.read().splitlines()
pd.Series(s_ex1)

0      Tile 2311:
1      ..##.#..#.
2      ##..#.....
3      #...##..#.
4      ####.#...#
          ...    
102    .#...#.##.
103    #.#####.##
104    ..#.###...
105    ..#.......
106    ..#.###...
Length: 107, dtype: object

In [379]:
orientations = range(0,8)

In [380]:
class Tile:
    """A simple tile class"""
    borders = []
    header = ""
    lines = []
    number = 0
    bordersForOrientation = []
    borderText = []

    def __init__(self, header, lines):
        self.header = header
        self.lines = lines
        self.initialize()

    def initialize(self):
        pattern = 'Tile (.*):'
        matches = re.match(pattern, self.header)
        self.number = int(matches.group(1))
        borderText = self.getBorderText(self.lines)
        self.borderText = borderText
        borders = []
        for text in borderText:
            borderValue = self.textToInt(text)            
            borders.append(borderValue)
        self.borders = borders
        bordersForOrientation = []
        for o in orientations:
            rotated = self.getBordersForOrientation(o)
            bordersForOrientation.append(rotated)
        self.bordersForOrientation = bordersForOrientation
    
    def textToInt(self, text):
        result = 0
        for c in text:
            result = (result << 1)
            if c == "#":
                result += 1
        return result

    def getBorderText(self, lines):        
        result = []                
        right = ""
        left = ""
        for row in lines:
            right = right + row[-1]
            left = row[0] + left
        result.append(lines[0])
        result.append(right)
        result.append(lines[-1][::-1])
        result.append(left)
        result.append(lines[0][::-1])
        result.append(right[::-1])
        result.append(lines[-1])
        result.append(left[::-1])        
        return result

    def getBordersForOrientation(self, orientation):        
        result = []
        #minorOffset = orientation % 4
        #offset = orientation - minorOffset
        #print(self.borders, orientation, minorOffset, offset)
        #result.extend(self.borders[minorOffset+offset:offset+4])
        #result.extend(self.borders[offset:offset+minorOffset])
        #return result
        if orientation == 0:
            return [self.borders[0],self.borders[1],self.borders[6],self.borders[7]]
        if orientation == 1:
            return [self.borders[1],self.borders[2],self.borders[7],self.borders[4]]
        if orientation == 2:
            return [self.borders[2],self.borders[3],self.borders[4],self.borders[5]]
        if orientation == 3:
            return [self.borders[3],self.borders[0],self.borders[5],self.borders[6]]
        if orientation == 4:
            return [self.borders[4],self.borders[7],self.borders[2],self.borders[1]]
        if orientation == 5:
            return [self.borders[7],self.borders[6],self.borders[1],self.borders[0]]
        if orientation == 6:
            return [self.borders[6],self.borders[5],self.borders[0],self.borders[3]]
        if orientation == 7:
            return [self.borders[5],self.borders[4],self.borders[3],self.borders[2]]
        return None

    def f(self):
        print("TILE",self.number)
        print(self.borders)
        for o in orientations:
            print("borders for orientation",o,self.getBordersForOrientation(o), self.bordersForOrientation[o])
        print("End")

In [381]:
def parse(input):
    result = {}
    header = ""
    lines = []
    for line in input:
        if line.count(":") > 0:
            header = line
        elif len(line) > 0:
            lines.append(line)
        else:
            tile = Tile(header, lines)
            result[tile.number] = tile
            lines = []
    if len(lines)>0:
        tile = Tile(header, lines)
        result[tile.number] = tile
    return result

In [382]:
p_ex1 = parse(s_ex1)
for t in p_ex1.values():
    t.f()
p = parse(s)

TILE 2311
[210, 89, 924, 318, 300, 616, 231, 498]
borders for orientation 0 [210, 89, 231, 498] [210, 89, 231, 498]
borders for orientation 1 [89, 924, 498, 300] [89, 924, 498, 300]
borders for orientation 2 [924, 318, 300, 616] [924, 318, 300, 616]
borders for orientation 3 [318, 210, 616, 231] [318, 210, 616, 231]
borders for orientation 4 [300, 498, 924, 89] [300, 498, 924, 89]
borders for orientation 5 [498, 231, 89, 210] [498, 231, 89, 210]
borders for orientation 6 [231, 616, 210, 318] [231, 616, 210, 318]
borders for orientation 7 [616, 300, 318, 924] [616, 300, 318, 924]
End
TILE 1951
[710, 498, 177, 587, 397, 318, 564, 841]
borders for orientation 0 [710, 498, 564, 841] [710, 498, 564, 841]
borders for orientation 1 [498, 177, 841, 397] [498, 177, 841, 397]
borders for orientation 2 [177, 587, 397, 318] [177, 587, 397, 318]
borders for orientation 3 [587, 710, 318, 564] [587, 710, 318, 564]
borders for orientation 4 [397, 841, 177, 498] [397, 841, 177, 498]
borders for orienta

In [383]:
def testNeighbors(data, details, x, y, direction):
    secondTileDetails = None
    if direction == 0:
        if x < len(details[0]) - 1:
            # check with right neighbor
            secondTileDetails = details[y][x+1]
        else:
            return True
    else:
        if y < len(details) - 1:
            # check with bottom neighbor
            secondTileDetails = details[y+1][x]
        else:
            return True    

    firstTileDetails = details[y][x]    
    firstTile = data[firstTileDetails[0]]
    secondTile = data[secondTileDetails[0]]    
    #secondTile.f()
    firstTileBorders = firstTile.bordersForOrientation[firstTileDetails[1]] 
    secondTileBorders = secondTile.bordersForOrientation[secondTileDetails[1]]    
    if direction == 0:
        print("TEST",x,y,direction, firstTileDetails, secondTileDetails, firstTileBorders, secondTileBorders, firstTileBorders[1], secondTileBorders[3])
        return firstTileBorders[1] == secondTileBorders[3]
    else:
        print("TEST",x,y,direction, firstTileDetails, secondTileDetails, firstTileBorders, secondTileBorders, firstTileBorders[2], secondTileBorders[0])
        return firstTileBorders[2] == secondTileBorders[0]

def testSolution(data, details):
    for x in range(0,len(details[0])):
        for y in range(0, len(details)):
            for d in range(0,2):
                testResult = testNeighbors(data, details, x, y, d)                
                if not testResult:
                    return False
    return True

In [446]:
details_ex1 = [[(1951,6),(2311,6),(3079,0)],[(2729,6),(1427,6),(2473,7)],[(2971,6),(1489,6),(1171,4)]]
details_ex1_list = [(1951,6),(2311,6),(3079,0),(2729,6),(1427,6),(2473,7),(2971,6),(1489,6),(1171,4)]

In [385]:
testSolution(p_ex1, details_ex1)

TEST 0 0 0 (1951, 6) (2311, 6) [564, 318, 710, 587] [231, 616, 210, 318] 318 318
TEST 0 0 1 (1951, 6) (2729, 6) [564, 318, 710, 587] [710, 9, 85, 962] 710 710
TEST 0 1 0 (2729, 6) (1427, 6) [710, 9, 85, 962] [210, 348, 948, 9] 9 9
TEST 0 1 1 (2729, 6) (2971, 6) [710, 9, 85, 962] [85, 689, 161, 78] 85 85
TEST 0 2 0 (2971, 6) (1489, 6) [85, 689, 161, 78] [948, 288, 848, 689] 689 689
TEST 1 0 0 (2311, 6) (3079, 0) [231, 616, 210, 318] [702, 264, 184, 616] 616 616
TEST 1 0 1 (2311, 6) (1427, 6) [231, 616, 210, 318] [210, 348, 948, 9] 210 210
TEST 1 1 0 (1427, 6) (2473, 7) [210, 348, 948, 9] [184, 481, 399, 348] 348 348
TEST 1 1 1 (1427, 6) (1489, 6) [210, 348, 948, 9] [948, 288, 848, 689] 948 948
TEST 1 2 0 (1489, 6) (1171, 4) [948, 288, 848, 689] [399, 902, 96, 288] 288 288
TEST 2 0 1 (3079, 0) (2473, 7) [702, 264, 184, 616] [184, 481, 399, 348] 184 184
TEST 2 1 1 (2473, 7) (1171, 4) [184, 481, 399, 348] [399, 902, 96, 288] 399 399


True

In [386]:
print(int("#...##.#..".replace(".","0").replace("#","1"),2))
print(int(".#..#####.".replace(".","0").replace("#","1"),2))
print(int(".##...##.#".replace(".","0").replace("#","1"),2))
print(int("##.#..#..#".replace(".","0").replace("#","1"),2))
print(int("#.##...##.".replace(".","0").replace("#","1"),2))
print(int("#..#..#.##".replace(".","0").replace("#","1"),2))

564
318
397
841
710
587


In [387]:
print(int("..###..###".replace(".","0").replace("#","1"),2))
print(int("#..##.#...".replace(".","0").replace("#","1"),2))
print(int(".#..#.##..".replace(".","0").replace("#","1"),2))
print(int(".#####..#.".replace(".","0").replace("#","1"),2))
print(int("..##.#..#.".replace(".","0").replace("#","1"),2))
print(int(".#..#####.".replace(".","0").replace("#","1"),2))

231
616
300
498
210
318


In [388]:
# 1171,4
print(".##...####", int(".##...####".replace(".","0").replace("#","1"),2))
print("####...##.", int("####...##.".replace(".","0").replace("#","1"),2))

print("###....##.", int("###....##.".replace(".","0").replace("#","1"),2))
print(".##....###", int(".##....###".replace(".","0").replace("#","1"),2))

print(".....##...", int(".....##...".replace(".","0").replace("#","1"),2))
print("...##.....", int("...##.....".replace(".","0").replace("#","1"),2))

print(".....#..#.", int(".....#..#.".replace(".","0").replace("#","1"),2))
print(".#..#.....", int(".#..#.....".replace(".","0").replace("#","1"),2))

.##...#### 399
####...##. 966
###....##. 902
.##....### 391
.....##... 24
...##..... 96
.....#..#. 18
.#..#..... 288


In [389]:
# 2311:
print("..##.#..#.", int("..##.#..#.".replace(".","0").replace("#","1"),2))
print(".#..#.##..", int(".#..#.##..".replace(".","0").replace("#","1"),2))

print("...#.##..#", int("...#.##..#".replace(".","0").replace("#","1"),2))
print("#..##.#...", int("#..##.#...".replace(".","0").replace("#","1"),2))

print("###..###..", int("###..###..".replace(".","0").replace("#","1"),2))
print("..###..###", int("..###..###".replace(".","0").replace("#","1"),2))

print(".#..#####.", int(".#..#####.".replace(".","0").replace("#","1"),2))
print(".#####..#.", int(".#####..#.".replace(".","0").replace("#","1"),2))

..##.#..#. 210
.#..#.##.. 300
...#.##..# 89
#..##.#... 616
###..###.. 924
..###..### 231
.#..#####. 318
.#####..#. 498


In [390]:
class PartOneTests(unittest.TestCase):
    def setUp(self):
        pass
    def tearDown(self):
        pass
    def test_tile_1171(self):
        tile = Tile("Tile 1171:",["####...##.", "#..##.#..#", "##.#..#.#.", ".###.####.", "..###.####", ".##....##.", ".#...####.", "#.##.####.", "####..#...",".....##..."])
        print(tile.borderText)
        self.assertEqual([966,288,24,902],tile.bordersForOrientation[0])
        self.assertEqual([288,96,902,399],tile.bordersForOrientation[1])
        self.assertEqual([96,391,399,18],tile.bordersForOrientation[2])
        self.assertEqual([391,966,18,24],tile.bordersForOrientation[3])
        self.assertEqual([399,902,96,288],tile.bordersForOrientation[4])
        self.assertEqual([902,24,288,966],tile.bordersForOrientation[5])
        self.assertEqual([24,18,966,391],tile.bordersForOrientation[6])
        self.assertEqual([18,399,391,96],tile.bordersForOrientation[7])
    def test_tile_2311(self):
        tile = Tile("Tile 2311:",["..##.#..#.","##..#.....","#...##..#.","####.#...#","##.##.###.","##...#.###",".#.#.#..##","..#....#..","###...#.#.","..###..###"])
        print(tile.borderText)
        self.assertEqual([210,89,231,498],tile.bordersForOrientation[0])
        self.assertEqual([89,924,498,300],tile.bordersForOrientation[1])
        self.assertEqual([924,318,300,616],tile.bordersForOrientation[2])
        self.assertEqual([318,210,616,231],tile.bordersForOrientation[3])
        self.assertEqual([300,498,924,89],tile.bordersForOrientation[4])
        self.assertEqual([498,231,89,210],tile.bordersForOrientation[5])
        self.assertEqual([231,616,210,318],tile.bordersForOrientation[6])
        self.assertEqual([616,300,318,924],tile.bordersForOrientation[7])
        for o in orientations:
            self.assertEqual(tile.bordersForOrientation[o], p_ex1[2311].bordersForOrientation[o] )
            self.assertEqual(tile.lines, p_ex1[2311].lines)


In [391]:
unittest.main(argv=[''], verbosity=1, exit=False)

..['####...##.', '.#..#.....', '...##.....', '.##....###', '.##...####', '.....#..#.', '.....##...', '###....##.']
['..##.#..#.', '...#.##..#', '###..###..', '.#..#####.', '.#..#.##..', '#..##.#...', '..###..###', '.#####..#.']

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [392]:
p_ex1[2311].f()

TILE 2311
[210, 89, 924, 318, 300, 616, 231, 498]
borders for orientation 0 [210, 89, 231, 498] [210, 89, 231, 498]
borders for orientation 1 [89, 924, 498, 300] [89, 924, 498, 300]
borders for orientation 2 [924, 318, 300, 616] [924, 318, 300, 616]
borders for orientation 3 [318, 210, 616, 231] [318, 210, 616, 231]
borders for orientation 4 [300, 498, 924, 89] [300, 498, 924, 89]
borders for orientation 5 [498, 231, 89, 210] [498, 231, 89, 210]
borders for orientation 6 [231, 616, 210, 318] [231, 616, 210, 318]
borders for orientation 7 [616, 300, 318, 924] [616, 300, 318, 924]
End


In [434]:
def findChoices(input, keys, leftDetails, topDetails):
    result = []
    for k in keys:
        tile = input[k]
        for o in orientations:
            borders = tile.bordersForOrientation[o]            
            if leftDetails != None:
                otherTile = input[leftDetails[0]]
                otherTileBorders = otherTile.bordersForOrientation[leftDetails[1]]
                if otherTileBorders[1] != borders[3]:
                    continue
            if topDetails != None:
                otherTile = input[topDetails[0]]
                otherTileBorders = otherTile.bordersForOrientation[topDetails[1]]
                if otherTileBorders[2] != borders[0]:
                    continue
            result.append( (k,o) )
    return result

def solveSubPuzzle(input, n, keys, solutionSoFar):
    if len(keys) == 0:
        return solutionSoFar
    newPosition = len(solutionSoFar)
    x = newPosition % n
    y = (newPosition - x)//n
    newIndex = (x,y)
    #print(solutionSoFar, keys, newIndex)
    leftDetails = None
    topDetails = None
    if x > 0:
        leftDetails = solutionSoFar[(y*n)+(x-1)]
    if y > 0:
        topDetails = solutionSoFar[((y-1)*n)+x]
    choices = findChoices(input, keys, leftDetails, topDetails)   
    currentSolution = solutionSoFar
    for c in choices:
        candidate = currentSolution[:]
        candidate.append(c)
        remainingKeys = keys[:]
        remainingKeys.remove(c[0])
        answer = solveSubPuzzle(input, n, remainingKeys, candidate)
        if answer != None:
            return answer
    return None

def solvePuzzle(input, n):
    keys = list(input.keys())
    print(keys)    
    choices = findChoices(input, keys, None, None)   
    print(choices)   
    currentSolution = []
    for c in choices:
        candidate = currentSolution[:]
        candidate.append(c)
        remainingKeys = keys[:]
        remainingKeys.remove(c[0])
        answer = solveSubPuzzle(input, n, remainingKeys, candidate)
        if answer != None:            
            corners = [answer[0][0],answer[n-1][0], answer[(n-1)*n][0], answer[n*n-1][0]]
            print(corners)
            product = 1
            for corner in corners:
                product *= corner
            print("Product is",product)
            return answer

In [435]:
solvePuzzle(p_ex1, 3)

[2311, 1951, 1171, 1427, 1489, 2473, 2971, 2729, 3079]
[(2311, 0), (2311, 1), (2311, 2), (2311, 3), (2311, 4), (2311, 5), (2311, 6), (2311, 7), (1951, 0), (1951, 1), (1951, 2), (1951, 3), (1951, 4), (1951, 5), (1951, 6), (1951, 7), (1171, 0), (1171, 1), (1171, 2), (1171, 3), (1171, 4), (1171, 5), (1171, 6), (1171, 7), (1427, 0), (1427, 1), (1427, 2), (1427, 3), (1427, 4), (1427, 5), (1427, 6), (1427, 7), (1489, 0), (1489, 1), (1489, 2), (1489, 3), (1489, 4), (1489, 5), (1489, 6), (1489, 7), (2473, 0), (2473, 1), (2473, 2), (2473, 3), (2473, 4), (2473, 5), (2473, 6), (2473, 7), (2971, 0), (2971, 1), (2971, 2), (2971, 3), (2971, 4), (2971, 5), (2971, 6), (2971, 7), (2729, 0), (2729, 1), (2729, 2), (2729, 3), (2729, 4), (2729, 5), (2729, 6), (2729, 7), (3079, 0), (3079, 1), (3079, 2), (3079, 3), (3079, 4), (3079, 5), (3079, 6), (3079, 7)]
[1951, 2971, 3079, 1171]
Product is 20899048083289


[(1951, 3),
 (2729, 3),
 (2971, 3),
 (2311, 3),
 (1427, 3),
 (1489, 3),
 (3079, 5),
 (2473, 2),
 (1171, 1)]

In [428]:
len(p)

144

In [483]:
solution_first_part = solvePuzzle(p, 12)

[3187, 1223, 1913, 1301, 3943, 3023, 3299, 3677, 2803, 3343, 1987, 1609, 2221, 2297, 2269, 2287, 2399, 1423, 2621, 2003, 3889, 1667, 1877, 2657, 1669, 2843, 2243, 1543, 1777, 2161, 3167, 1279, 2711, 1087, 3947, 1571, 2521, 3533, 2339, 1493, 1637, 2153, 1129, 3581, 1303, 3209, 3613, 3833, 2089, 1553, 2213, 3347, 3203, 1811, 1531, 3191, 3259, 1579, 1861, 2857, 3853, 3917, 1999, 3719, 3491, 3499, 2311, 1459, 1447, 2579, 2531, 1019, 1151, 3767, 1871, 2953, 1153, 3359, 3257, 2557, 2851, 1103, 1801, 2731, 2111, 1613, 3739, 1297, 2029, 1093, 2357, 1231, 2671, 3137, 1229, 1979, 2341, 1373, 1483, 3109, 3449, 2969, 3877, 3539, 3547, 3863, 2437, 2267, 3671, 3041, 2467, 3593, 3061, 1409, 2207, 1069, 1187, 2549, 2593, 1277, 2251, 1061, 3037, 3823, 2087, 1321, 1951, 3847, 3851, 3407, 1709, 1097, 3301, 3517, 2137, 3623, 1867, 1787, 2591, 1607, 3271, 2063, 1289, 1213]
[(3187, 0), (3187, 1), (3187, 2), (3187, 3), (3187, 4), (3187, 5), (3187, 6), (3187, 7), (1223, 0), (1223, 1), (1223, 2), (1223, 3), (1

In [464]:
def applyMirror(lines):
    result = []
    for line in lines:
        result.append(line[::-1])
    return result

def applyAntiClockwise(lines):
    n = len(lines)
    result = []
    for y in range(0,n):
        newLine = ""
        for x in range(0,n):
            newLine = newLine + lines[x][n-y-1]
        result.append(newLine)
    return result

def applyRotation(lines, o):
    result = []
    if o == 1:
        return applyAntiClockwise(lines)
    elif o == 2:
        return applyMirror(lines)[::-1]
    elif o == 3:
        return applyMirror(applyAntiClockwise(lines))[::-1]
    else:
        return lines[:]

def applyOrientation(lines, o):
    if o >= 4:
        return applyRotation(applyMirror(lines), o%4)
    else:
        return applyRotation(lines, o)

def trimLeft(lines):
    result = []
    for line in lines:
        result.append[line[1:]]
    return result

def trimTop(lines):
    result = []
    for line in lines[1:]:
        result.append(line)
    return result

def trimBorders(lines):
    result = []
    for line in lines[1:-1]:
        result.append(line[1:-1])
    return result

def printLines(lines):
    for line in lines:
        print(line)

In [445]:
for o in orientations:
    print ("ORIENTATION", o)
    tile = p_ex1[2311]
    printLines(applyOrientation(tile.lines,o))

ORIENTATION 0
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###
ORIENTATION 1
...#.##..#
#.#.###.##
....##.#.#
....#...#.
#.##.##...
.##.#....#
#..##.#..#
#..#...###
.#.####.#.
.#####..#.
ORIENTATION 2
###..###..
.#.#...###
..#....#..
##..#.#.#.
###.#...##
.###.##.##
#...#.####
.#..##...#
.....#..##
.#..#.##..
ORIENTATION 3
.#..#####.
.#.####.#.
###...#..#
#..#.##..#
#....#.##.
...##.##.#
.#...#....
#.#.##....
##.###.#.#
#..##.#...
ORIENTATION 4
.#..#.##..
.....#..##
.#..##...#
#...#.####
.###.##.##
###.#...##
##..#.#.#.
..#....#..
.#.#...###
###..###..
ORIENTATION 5
.#####..#.
.#.####.#.
#..#...###
#..##.#..#
.##.#....#
#.##.##...
....#...#.
....##.#.#
#.#.###.##
...#.##..#
ORIENTATION 6
..###..###
###...#.#.
..#....#..
.#.#.#..##
##...#.###
##.##.###.
####.#...#
#...##..#.
##..#.....
..##.#..#.
ORIENTATION 7
#..##.#...
##.###.#.#
#.#.##....
.#...#....
...##.##.#
#....#.##.
#..#.##..#
###...#..#
.#.####.#.
.#..#####.


In [467]:
def mergeTiles(input, solution, n):
    result = []
    for y in range(0,n):
        mergedLines = []
        for x in range(0,n):
            details = solution[x+y*n]
            tile = input[details[0]]
            part = trimBorders(applyOrientation(tile.lines, details[1]))

            #print ("GOT A PART" , details)
            #printLines(part)

            newLines = []
            if x == 0:
                for line in part:
                     newLines.append(line)
            else:
                for i in range(0,len(mergedLines)):
                    newLines.append(mergedLines[i] + part[i])

            mergedLines = newLines
        result.extend(mergedLines)
    return result

In [468]:
merged_ex1_solution = mergeTiles(p_ex1,details_ex1_list,3)
printLines(merged_ex1_solution)

.#.#..#.##...#.##..#####
###....#.#....#..#......
##.##.###.#.#..######...
###.#####...#.#####.#..#
##.#....#.##.####...#.##
...########.#....#####.#
....#..#...##..#.#.###..
.####...#..#.....#......
#..#.##..#..###.#.##....
#.####..#.####.#.#.###..
###.#.#...#.######.#..##
#.####....##..########.#
##..##.#...#...#.#.#.#..
...#..#..#.#.##..###.###
.#.#....#.##.#...###.##.
###.#...#..#.##.######..
.#.#.###.##.##.#..#.##..
.####.###.#...###.#..#.#
..#.#..#..#.#.#.####.###
#..####...#.#.#.###.###.
#####..#####...###....##
#.##..#..#...#..####...#
.#.###..##..##..####.##.
...###...##...#...#..###


In [469]:
seaMonsterLines = ["                  # ","#    ##    ##    ###"," #  #  #  #  #  #   "]

def parseSeaMonster(input):
    result = []
    for y in range(0,len(input)):
        line = input[y]
        for x in range(0,len(line)):
            if line[x] == "#":
                result.append((x,y))
    return result

seaMonsterIndexes = parseSeaMonster(seaMonsterLines)
seaMonsterIndexes

[(18, 0),
 (0, 1),
 (5, 1),
 (6, 1),
 (11, 1),
 (12, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (1, 2),
 (4, 2),
 (7, 2),
 (10, 2),
 (13, 2),
 (16, 2)]

In [477]:
def matchesSeaMonster(input, x, y):
    n = len(input)
    for i in seaMonsterIndexes:
        targetX = x + i[0]
        targetY = y + i[1]
        if targetX >= n:
            return False
        if targetY >= n:
            return False
        if input[targetY][targetX] == ".":
            return False
    return True

def markSeaMonster(input, x, y):
    result = input[:]
    for i in seaMonsterIndexes:
        targetX = x + i[0]
        targetY = y + i[1]
        targetLine = result[targetY]
        result[targetY] = targetLine[:targetX] + "O" + targetLine[targetX + 1:]        
    return result

def replaceSeaMonsters(input):
    result = input[:]
    count = 0
    for x in range(0,len(input[0])):
        for y in range(0,len(input)):
            if (matchesSeaMonster(input, x, y)):
                result = markSeaMonster(result, x, y)
                count += 1
    return (result, count)

In [481]:
def solvePartTwo(input):
    for o in orientations:
        orientedImage = applyOrientation(input, o)
        orientedImageWithSeaMonsters = replaceSeaMonsters(orientedImage)
        if orientedImageWithSeaMonsters[1] > 0:
            print (orientedImageWithSeaMonsters[1])
            printLines(orientedImageWithSeaMonsters[0])
            echoCount = 0
            for line in orientedImageWithSeaMonsters[0]:
                echoCount += line.count("#")
            return echoCount

In [482]:
solvePartTwo(merged_ex1_solution)

2
.####...#####..#...###..
#####..#..#.#.####..#.#.
.#.#...#.###...#.##.O#..
#.O.##.OO#.#.OO.##.OOO##
..#O.#O#.O##O..O.#O##.##
...#.#..##.##...#..#..##
#.##.#..#.#..#..##.#.#..
.###.##.....#...###.#...
#.####.#.#....##.#..#.#.
##...#..#....#..#...####
..#.##...###..#.#####..#
....#.##.#.#####....#...
..##.##.###.....#.##..#.
#...#...###..####....##.
.#.##...#.##.#.#.###...#
#.###.#..####...##..#...
#.###...#.##...#.##O###.
.O##.#OO.###OO##..OOO##.
..O#.O..O..O.#O##O##.###
#.#..##.########..#..##.
#.#####..#.#...##..#....
#....##..#.#########..##
#...#.....#..##...###.##
#..###....##.#...##.##.#


273

In [484]:
merged_solution = mergeTiles(p,solution_first_part,12)

In [485]:
solvePartTwo(merged_solution)

19
###....#..#..#.#.....#......#.#......#.....#....#....#.#...##...........##...##.#.........##..#.
.##....#.....#..#.#.#.........#....####...#.........#.......##...#...##...#.#........#....#....#
...#....#.#.......##.....###......#..#..#......#....#....#...#..#..#....##.......#...#.#.#......
....##.#.##.#..#..........##..##.#.#.#.#..#.#..#....#...#...#..##..#.#..##.........#.#..#...#..#
#.#.##.#...#......#.........#....##.#.#.#...#..#...#.....#.#....##.....###...#......#...#...##..
#..#.....##.......#..#.......#......#.#......#.........##..#....#...##.....#....#...........##..
.....................#...#.......#....#.##.#...#.....##.....##..#...#.#...#.........#.#.....#.##
..#...#....#..#..##..##..#....#...#.......##...##......#...#......#.#....#..#.......#.#..#.....#
...##.#......#...#...#.........#...#....##..##...##..#.####..#......##..O....#.#......#...##.#.#
.......#.#...###....#.O#..#..#...#..........#..#......O#...OO....OO#...OOO............#.#....#..
#...O...#OO..##OO..#.OOO...

2304